# Consultas 

## Líbrerías a ocupar 

In [2]:
import numpy as np
import pandas as pd

In [9]:
plebiscitoAR = pd.read_csv("data/Plebiscito Apruebo-Rechazo.csv", sep=";", decimal=",")
plebiscitoC = pd.read_csv("data/Plebiscito convención.csv", sep=";", decimal=",")
presidencial = pd.read_csv("data/resultadosComuna.csv", sep=";", decimal=",")